In [1]:
from tqdm import tqdm
from analysis.nlp_transformations import separate_previous_conversation, remove_footers
from analysis.attribute_retriving import stylo_metrix_analysis
from typing import List
from dao.email import DAOEmailGenerated, DAORealEmail

from dao.attribute import DAOAttribute
from analysis.nlp_transformations import replace_links_with_text

[nltk_data] Downloading package stopwords to /home/pawel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pawel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package pl196x to /home/pawel/nltk_data...
[nltk_data]   Package pl196x is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pawel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
text = "Oto kilkanaście zdań w języku polskim, które zawierają różnorodne formy interpunkcyjne i struktury zdaniowe. Jak się dzisiaj czujesz? Dlaczego niebo jest niebieskie? Czy wiesz, że Ks. Jan Twardowski był znanym polskim poetą? Uwaga! Proszę nie biec po korytarzu. Gdzie jest najbliższa apteka? Ile to kosztuje? To niesamowite, że możemy rozmawiać online! Co sądzisz o najnowszych odkryciach w kosmosie? Pamiętaj, że drzwi otwierają się o godz. 8:00 rano! Czy możesz mi podać sól? Kto zostanie nowym prezydentem Stanów Zjednoczonych? Wow! To było naprawdę ekscytujące. Jak możemy efektywniej oszczędzać energię? Czy znasz jakieś dobre przepisy na ciasto? Dr M. Nowak będzie dziś wygłaszać wykład na uniwersytecie. Czy myślisz, że sztuczna inteligencja może przewyższyć ludzką kreatywność?"
stylo_metrix_analysis([text], "pl")

/home/pawel/.virtualenvs/anti-gpt-checker/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'pl_nask' (0.0.7) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
100%|██████████| 1/1 [00:00<00:00,  2.87it/s]


[AllStyloMetrixFeaturesPL(text='Oto kilkanaście zdań w języku polskim, które zawierają różnorodne formy interpunkcyjne i struktury zdaniowe. Jak się dzisiaj czujesz? Dlaczego niebo jest niebieskie? Czy wiesz, że Ks. Jan Twardowski był znanym polskim poetą? Uwaga! Proszę nie biec po korytarzu. Gdzie jest najbliższa apteka? Ile to kosztuje? To niesamowite, że możemy rozmawiać online! Co sądzisz o najnowszych odkryciach w kosmosie? Pamiętaj, że drzwi otwierają się o godz. 8:00 rano! Czy możesz mi podać sól? Kto zostanie nowym prezydentem Stanów Zjednoczonych? Wow! To było naprawdę ekscytujące. Jak możemy efektywniej oszczędzać energię? Czy znasz jakieś dobre przepisy na ciasto? Dr M. Nowak będzie dziś wygłaszać wykład na uniwersytecie. Czy myślisz, że sztuczna inteligencja może przewyższyć ludzką kreatywność?', grammatical_forms=GrammaticalFormsPL(G_N=0.17857142857142858, G_V=0.18571428571428572, G_ADJ=0.12142857142857143, G_ADV=0.06428571428571428, G_PRO=0.11428571428571428, G_PRO_PRS=0.

In [3]:
from models.attribute import AttributeInDB

dao_generated_emails: DAOEmailGenerated = DAOEmailGenerated()
dao_real_emails: DAORealEmail = DAORealEmail()
dao_attribute: DAOAttribute = DAOAttribute()
english_batch = []
polish_batch = []

all_attributes: List[AttributeInDB] = dao_attribute.find_many_by_query({'stylometrix_metrics': None})

for attribute in tqdm(all_attributes, total=len(all_attributes), desc=f'Calculating stylometrix attributes', unit='emails', miniters=1):
    if attribute.referenced_db_name == 'email_generated_dataset':
        og_email = dao_generated_emails.find_one_by_query({'_id': attribute.referenced_doc_id})
        language = og_email.language
    else:
        og_email = dao_real_emails.find_one(attribute.referenced_db_name, {'_id': attribute.referenced_doc_id})
        language = og_email.detected_language
    
    _, og_text = separate_previous_conversation(og_email.text_plain)
    og_text = remove_footers(og_text)
    og_text = replace_links_with_text(og_text, "")
    
    if language == "pl":
        polish_batch.append([attribute.id, og_text])
    else:
        english_batch.append([attribute.id, og_text])
        
    stylometrix_metrics = None
    batch_w_results = None
    if len(polish_batch) >= 20:
        batch_w_results = polish_batch
        stylometrix_metrics = stylo_metrix_analysis([batch[1] for batch in polish_batch], 'pl')
        polish_batch = []
    elif len(english_batch) >= 20:
        batch_w_results = english_batch
        stylometrix_metrics = stylo_metrix_analysis([batch[1] for batch in english_batch], 'en')
        english_batch = []
        
    if stylometrix_metrics is not None:
        for attr_id, sm_results in zip(batch_w_results, stylometrix_metrics):
            dao_attribute.update_one({'_id': attr_id[0]}, {'$set':{'stylometrix_metrics': sm_results.dict()}})
            
if len(polish_batch) > 0:
    stylometrix_metrics = stylo_metrix_analysis([batch[1] for batch in polish_batch], 'pl')
    for attr_id, sm_results in zip(polish_batch, stylometrix_metrics):
            dao_attribute.update_one({'_id': attr_id[0]}, {'$set':{'stylometrix_metrics': sm_results.dict()}})
        
if len(english_batch) > 0:
    stylometrix_metrics = stylo_metrix_analysis([batch[1] for batch in english_batch], 'en')
    for attr_id, sm_results in zip(english_batch, stylometrix_metrics):
            dao_attribute.update_one({'_id': attr_id[0]}, {'$set':{'stylometrix_metrics': sm_results.dict()}})

 75%|███████▌  | 15/20 [00:05<00:01,  3.69it/s]

float division by zero
 AT METRIC ST_HERDAN_TTR, TEXT: /columnists/20020905brian5.asp...



 65%|██████▌   | 13/20 [00:04<00:01,  3.51it/s]

float division by zero
 AT METRIC ST_HERDAN_TTR, TEXT: /columnists/20020905brian5.asp...



 75%|███████▌  | 15/20 [00:05<00:01,  3.57it/s]

float division by zero
 AT METRIC ST_HERDAN_TTR, TEXT: /columnists/20020905brian5.asp...



 25%|██▌       | 5/20 [00:00<00:03,  4.94it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (879 > 512). Running this sequence through the model will result in indexing errors

 15%|█▌        | 3/20 [00:00<00:05,  3.33it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1296 > 512). Running this sequence through the model will result in indexing errors

 35%|███▌      | 7/20 [00:02<00:04,  2.85it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1744 > 512). Running this sequence through the model will result in indexing errors

 95%|█████████▌| 19/20 [00:02<00:00,  4.91it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1114 > 512). Running this sequence through the model will result in indexing errors

 45%|████▌     | 9/20 [00:01<00:01,  5.91it/s]Token indices sequence length is 